# In this notebook i would like to demonstrate the MLP classifier and also high level API model Keras for my cryptotheraphy dataset and explore the results of cost metrics to see how well these models are performing to my dataset and compare with my previous model in Assignment-1 notebbok.


# Description of the data set:

The data set is about CRYPTOTHERAPHY ANALYSIS.This data set provides patients characterstics after the tratment the level of cancer.
It states whether the cancer level is benign(1) or malignant(0). This data set contains the following attributes:
1. SEX- It indicates the gender of the patient whetehrthe patient is a male or female.(1=male,2=female)
2. AGE- It indicates the age of the patient.
3. Time- This column indicates the time for the treatment in seconds.
4. Number of warts- This column indicates the numbe rof warts that are grown on a patients body. ( Warts are skin growths caused by a virus. The virus infects the top layer of skin, causing it to grow rapidly.Warts can grow anywhere on your body. Most warts go away on their own, but they may come back.)
5. Type : It indicates the type of wart. ( there are mainly three types of warts . 1,2,3 ( common warts, flat,filtform)
6. Area: The composition of wart .
7. Result of the tratment: It indicates the level of cancer after tratment.

The target variable is Result of the tratment. It is a binary classifier. benign(1) or malignant(0).
This data set is a classification type of problem.
The data set contains the instances of the 90 patients that are obtained from the results of cryptotheraphy treatment.

Source of the data set: It is taken from the Mashhad University of Medical Sciences, Mashhad, Iran. Fahime Khozeimeh, MD, Pouran Layegh, Professor of Dermatology and Roohallah Alizadehsani, PhD student and Mohamad Roshanzamir, PhD candidate.

It is published in UCI MACHINE LEARNING REPOSITORY: https://archive.ics.uci.edu/ml/datasets/Cryotherapy+Dataset+


# Business problem statement and its purpose:
1. The analysis of this data set will identify the results of the cancer treatment. It is a health care industry domain.
2. Purpose :  This will helps the to explore and draw some inferences that may help the people to undergo the Cryotherapic treatment.
3. It also helps the crytotheraphists to see the sttaistics and how well their treatment serves the people. How they can improve the level of treatment.
4. It also helps to identify what age groups of patients  the treatment is getting malignant and what age groups of people is getting benign 
5. In this notebook I would like to do data preprocessing that is ready to apply modleing .
6. For this assignment I would like to use the following algorithms (Discussed in class so far) to identify how well my models are trained and see how it performs of test data and if a new data is given it can able to predict the likelihood of results of level of tretament .


## In my assignment-1 data prep notebook I already done the data preprocessing and saved them to files and I am using those files directly to load and fit my models.

# Load libraries

In [1]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
from __future__ import print_function
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn import datasets


np.random.seed(1)

# Load data

In [3]:
X_train = pd.read_csv("crypto_data_train_X.csv")
X_test = pd.read_csv("crypto_data_test_X.csv")
y_train = pd.read_csv("crypto_data_train_y.csv")
y_test = pd.read_csv("crypto_data_test_y.csv")

In [11]:
X_train.shape

(63, 6)

# Neural Network with MLP classifier

In [5]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(40,30,20), solver='adam', max_iter=200)
_ = ann.fit(X_train, y_train)

CPU times: total: 250 ms
Wall time: 176 ms


In [57]:
%%time
y_pred = ann.predict(X_test)

CPU times: total: 0 ns
Wall time: 2 ms


In [58]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.67      0.77        15
           1       0.69      0.92      0.79        12

    accuracy                           0.78        27
   macro avg       0.80      0.79      0.78        27
weighted avg       0.81      0.78      0.78        27



# F1 metric using Randomised search cv for neural net using MLP classifier

In [59]:
%%time
from sklearn.metrics import f1_score, make_scorer
score_measure = make_scorer(f1_score , average='macro')
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (90,), (80,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['logistic', 'tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.01, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (60, 40, 20), 'alpha': 0.2, 'activation': 'logistic'}
CPU times: total: 2.03 s
Wall time: 15.1 s


In [60]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.73      0.85        15
           1       0.75      1.00      0.86        12

    accuracy                           0.85        27
   macro avg       0.88      0.87      0.85        27
weighted avg       0.89      0.85      0.85        27

CPU times: total: 0 ns
Wall time: 11 ms


#  grid search cv for neural net for MLP classifier

In [61]:
%%time

score_measure = make_scorer(f1_score , average='macro')
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (30,), (50,), (70,), (90,)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam'],
    'alpha': [.5, .7, 1],
    'learning_rate': ['adaptive', 'invscaling'],
    'learning_rate_init': [0.005, 0.01, 0.15],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = GridSearchCV(estimator = ann, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
{'activation': 'tanh', 'alpha': 0.7, 'hidden_layer_sizes': (30,), 'learning_rate': 'adaptive', 'learning_rate_init': 0.01, 'max_iter': 5000, 'solver': 'adam'}
CPU times: total: 578 ms
Wall time: 17.4 s


In [62]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.73      0.81        15
           1       0.73      0.92      0.81        12

    accuracy                           0.81        27
   macro avg       0.82      0.82      0.81        27
weighted avg       0.84      0.81      0.81        27

CPU times: total: 0 ns
Wall time: 11 ms


# Using Keras neural network model

In [6]:
import tensorflow as tf
from tensorflow import keras

# fix random seed for reproducibility
np.random.seed(1)
tf.random.set_seed(1)

In [9]:
%%time

# create model stucture
model = keras.models.Sequential()
model.add(keras.layers.Input(6))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(30, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid')) # final layer, 1 categories

import keras.backend as K

def f1_score(y_test, y_pred):
    """
    Custom metric for F1 score.
    
    Arguments:
    y_test -- true labels
    y_pred -- predicted labels
    
    Returns:
    f1_score -- F1 score
    """
    # Calculate true positives, false positives, and false negatives
    tp = K.sum(K.round(K.clip(y_test * y_pred, 0, 1)))
    fp = K.sum(K.round(K.clip(y_pred - y_test, 0, 1)))
    fn = K.sum(K.round(K.clip(y_test - y_pred, 0, 1)))

    # Calculate precision, recall, and F1 score
    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())

    return f1_score

# compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1_score])

# if you want to overide the defaults for the optimizer....
#adam = keras.optimizers.Adam(learning_rate=0.01)
#model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


CPU times: total: 15.6 ms
Wall time: 42.6 ms


In [10]:
%%time

# fit the model
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, batch_size=10)


Epoch 1/10
Cause: Unable to locate the source code of <function f1_score at 0x000001FEE095FE50>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: Unable to locate the source code of <function f1_score at 0x000001FEE095FE50>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the fun

In [55]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores
# In results, first is loss, second is accuracy

[0.6576361060142517, 0.3157894015312195]

In [56]:
# let's format this into a better output...

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.66
f1_score: 31.58%


# Wide and Deep Network

In [29]:
#Define the model: for multi-class

model = keras.models.Sequential()

model.add(keras.layers.Input(shape=6))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

In [30]:
# Compile model

#Optimizer:
adam = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=[f1_score])

In [31]:
# Fit the model

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    epochs=10, batch_size=10)

Epoch 1/10
7/7 [==============================] - 1s 52ms/step - loss: 8.1214 - f1_score: 0.2658 - val_loss: 1.0076 - val_f1_score: 0.0000e+00
Epoch 2/10
7/7 [==============================] - 0s 8ms/step - loss: 2.0248 - f1_score: 0.4486 - val_loss: 2.2648 - val_f1_score: 0.6127
Epoch 3/10
7/7 [==============================] - 0s 7ms/step - loss: 0.9955 - f1_score: 0.5268 - val_loss: 1.6088 - val_f1_score: 0.0000e+00
Epoch 4/10
7/7 [==============================] - 0s 7ms/step - loss: 1.2522 - f1_score: 0.2492 - val_loss: 1.4346 - val_f1_score: 0.5238
Epoch 5/10
7/7 [==============================] - 0s 7ms/step - loss: 0.9337 - f1_score: 0.7880 - val_loss: 0.6737 - val_f1_score: 0.0000e+00
Epoch 6/10
7/7 [==============================] - 0s 7ms/step - loss: 0.6570 - f1_score: 0.3912 - val_loss: 0.7684 - val_f1_score: 0.5833
Epoch 7/10
7/7 [==============================] - 0s 7ms/step - loss: 0.6462 - f1_score: 0.7225 - val_loss: 0.7141 - val_f1_score: 0.3148
Epoch 8/10
7/7 [=====

In [32]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.646960437297821, 0.615384578704834]

In [33]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

loss: 0.65
f1_score: 61.54%


# RandomGridSearch

In [34]:
%%time

# If you don't have the following installed, from command line '!pip install scikeras'
from scikeras.wrappers import KerasClassifier
from keras.initializers import GlorotNormal

score_measure = f1_score
kfolds = 5

def build_clf(hidden_layer_sizes, dropout):
    ann = tf.keras.models.Sequential()
    ann.add(keras.layers.Input(shape=6)),
    for hidden_layer_size in hidden_layer_sizes:
        model.add(keras.layers.Dense(hidden_layer_size, kernel_initializer= tf.keras.initializers.GlorotNormal(), 
                                     bias_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None), activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    ann.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    ann.compile(loss = 'binary_crossentropy', metrics = [f1_score])
    return ann


CPU times: total: 0 ns
Wall time: 0 ns


In [35]:
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_clf,
    hidden_layer_sizes=7,
    dropout = 0.0
)


In [41]:
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV

params = {
    'optimizer__learning_rate': [0.01],
    'model__hidden_layer_sizes': [(700,),(700,500)],
    'model__dropout': [0, 0.1],
    'batch_size':[2000],
    'epochs':[2000],
    'optimizer':["adam"]
}
keras_clf.get_params().keys()



dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs', 'hidden_layer_sizes', 'dropout', 'class_weight'])

In [42]:
rnd_search_cv = RandomizedSearchCV(estimator=keras_clf, param_distributions=params, scoring='accuracy', n_iter=50, cv=5)

import sys
sys.setrecursionlimit(10000) # note: the default is 3000 (python 3.9)

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
callback = [earlystop]

_ = rnd_search_cv.fit(X_train, y_train, callbacks=callback, verbose=0)


1/1 [==============================] - 0s 58ms/step


In [43]:
rnd_search_cv.best_params_

{'optimizer__learning_rate': 0.01,
 'optimizer': 'adam',
 'model__hidden_layer_sizes': (700, 500),
 'model__dropout': 0.1,
 'epochs': 2000,
 'batch_size': 2000}

In [44]:
best_net = rnd_search_cv.best_estimator_
print(rnd_search_cv.best_params_)

{'optimizer__learning_rate': 0.01, 'optimizer': 'adam', 'model__hidden_layer_sizes': (700, 500), 'model__dropout': 0.1, 'epochs': 2000, 'batch_size': 2000}


In [45]:
%%time
y_pred = best_net.predict(X_test)
print(classification_report(y_test, y_pred))

1/1 [==============================] - 0s 44ms/step
              precision    recall  f1-score   support

           0       0.69      0.60      0.64        15
           1       0.57      0.67      0.62        12

    accuracy                           0.63        27
   macro avg       0.63      0.63      0.63        27
weighted avg       0.64      0.63      0.63        27

CPU times: total: 62.5 ms
Wall time: 108 ms


# Analysis:
1. As per the assignment I have developed Neural network model using MLP classifier and also Keras API.
2. For my cryptotheraphy dataset the results for Neural network using MLP classifier and Keras API are as follows.
3. Remember here for my dataset I am considering F1 score as best metric which I argued in assignment-1.
4. Lets compare the f1 scores for neural network model using MLP classifier and keras api before and after hyper parameter tuning.
5. For keras API i intially I started with less neurons and then increased the neurons.
6. Lets compare:
                                         
                                         
        MLP CLASSIFIER  Before tuning    79   
        
        After tuning  Random search      86 
        
                      Grid search        81
        
        Keras with less neurons         31.58  
        
        Keras with more neurons         61.54 
        
        After tuning  it is 64%
                                      
 I tried selu, elu, with he_normal,lecun_normal around 2 days some times it is giving 78, 85 and some times it is underfitting.
    
How ever among these both MLP classifier after hyper parameter tuning is having best results of about 86 percent.
    
 Now comparing this with my old assignment 1 models Among my previous models decision tree is best model with a f1 score value of about 92.11.
    
 So, deciison tree is performing well when compared to the neural networks.
    
 
 So, I can say that for my data set F1 score is best metric and Deciison tree model is best model (As far as the models which i run till now from      assignment 1, assignment 2)
                                         